# Breakpoint Analysis Through Segmented Regression

In [ ]:
import os

import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from prophet import Prophet

import piecewise_regression as pr

import statsmodels.formula.api as smf

import plotly.graph_objects as go

import pymannkendall as mk

import tqdm

In [ ]:
data_folder = os.path.join("..", "data", "berlin")
clean_data_folder = os.path.join(data_folder, "clean")

In [ ]:
surface_df = pd.read_excel(
    os.path.join(clean_data_folder, "berlin.xlsx")
)

In [ ]:
columns = surface_df.columns.difference(['DateTime', 'Station'])

In [ ]:
seed = 21597

## Find Breakpoints 

In [ ]:
station_305_df = surface_df[surface_df['Station'] == 305].copy()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
results_305_df = pd.DataFrame(
    index=columns,
    columns=pd.MultiIndex.from_product([[0, 1, 2], ['R2 Adj', 'Model']])
)

In [ ]:
# %%script false --no-raise-error
# this will require around 40 minutes to run      

for column in tqdm.notebook.tqdm(columns):
    
    df = station_305_df[['DateTime', column]].copy()
    
    df.dropna(inplace=True)
    
    df['DateTime'] = df['DateTime'].astype(int)
    df['DateTime'] = scaler.fit_transform(df[['DateTime']])
    
    
    for i in range(3):
        
        # classic linear regression
        if i == 0:
            
            X, y = df['DateTime'].values.reshape(-1, 1), df[column].values
            
            model = smf.ols(f'Q("{column}") ~ DateTime', data=df).fit()
            results_305_df.loc[column, (i, 'R2 Adj')] = model.rsquared_adj
            results_305_df.loc[column, (i, 'Model')] = model
            
        else:
            
            best_r2 = -np.inf
            best_model = 'No Convergence'
            
            
            np.random.seed(seed)
            
            # segmented regression
            X, y = df['DateTime'].values, df[column].values
            
            model = pr.Fit(X, y, n_breakpoints=i, max_iterations=1000)
            
            if model.get_results()['converged']:
                r2 = model.best_muggeo.best_fit.adjusted_r_squared
                
                if r2 > best_r2:
                    best_r2 = r2
                    best_model = model
                
            
            results_305_df.loc[column, (i, 'R2 Adj')] = best_r2
            results_305_df.loc[column, (i, 'Model')] = best_model

In [ ]:
results_305_df

In [ ]:
def set_y_scale(parameter):
    """
    Set the y-axis scale for the plot based on the parameter.
    """
    if parameter == 'Air Temperature (°C)':
        return [-10, 35]
    elif parameter == 'Cumulated Rainfall (mm)':
        return [0, 7]
    elif parameter == 'DOC (mg/l)':
        return [4, 14]
    elif parameter == 'Nitrate (mg/l)':
        return [0, 6]
    elif parameter == 'Ammonium (mg/l)':
        return [0, 4]
    elif parameter == 'Conductivity (µS/cm)':
        return [250, 1100]
    elif parameter == 'Dissolved Oxygen (mg/l)':
        return [3, 23]
    elif parameter == 'Flow River Rate (m³/s)':
        return [0, 160]
    elif parameter == 'Sulphate (mg/l)':
        return [10, 240]
    elif parameter == 'pH':
        return [6.5, 9]
    elif parameter == 'Water Temperature (°C)':
        return [0, 30]

In [ ]:
for parameter in results_305_df.index:
    
    print(f'Parameter: {parameter}')
    
    r2_scores = results_305_df.loc[parameter, (slice(None), 'R2 Adj')]
    
    r2_scores = r2_scores.apply(lambda x: -np.inf if isinstance(x, str) else x)
    best_model_index = r2_scores.idxmax()[0]
    # best_model_index = 2
    best_model = results_305_df.loc[parameter, (best_model_index, 'Model')]
    
    df = station_305_df[['DateTime', parameter]].copy()
    df.dropna(inplace=True)
    
    datetime_column = df['DateTime'].copy()
    
    start_year = df['DateTime'].dt.year.min()
    end_year = df['DateTime'].dt.year.max()
    tickvals = [pd.Timestamp(f'{year}-02-26') for year in range(start_year, end_year + 1, 2)]
    ticktext = [str(year) for year in range(start_year, end_year + 1, 2)]   

    
    df['DateTime'] = df['DateTime'].astype(int)
    df['DateTime'] = scaler.fit_transform(df[['DateTime']])
    
    # ols model
    if best_model_index == 0:
        
        # plot the model
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=datetime_column, y=df[parameter], mode='lines', line=dict(color='black', width=1.5), opacity=0.5))
        fig.add_trace(go.Scatter(x=datetime_column, y=best_model.predict(df['DateTime']), mode='lines', line=dict(color='rgb(200, 2, 110)', width=2.0)))
        
        fig.update_xaxes(
            tickvals=tickvals,
            ticktext=ticktext,
            title_text="Time",
            tickangle=90  # Add vertical rotation to x-axis labels
        )
        
        parameter_ = parameter.replace('/', '_')
        
        scale = set_y_scale(parameter)
        
        fig.update_yaxes(
            range=scale
        )
        
        fig.update_layout(
            xaxis_title='Time',
            yaxis_title=parameter,
            showlegend=False,
            margin=dict(l=10, r=10, t=10, b=10),
            font=dict(size=20)
        )
        
        fig.write_image(
            f'plots/305/{parameter_}.png',
            scale=3
        )
        
        
        # fig.show()
        
    # piecewise model
    else:
        
        if best_model_index == 1:

            br_value = round(best_model.get_results()['estimates']['breakpoint1']['estimate'], 6)
            datetime_value = datetime_column[(df['DateTime'] - br_value).abs().idxmin()]
            
            print(f'R2: {r2_scores.max()}')
            print(f'Breakpoint: {datetime_value}')
            
            fig = go.Figure()
            
            fig.add_trace(go.Scatter(x=datetime_column, y=df[parameter], mode='lines', line=dict(color='black', width=1.5), opacity=0.5))
            fig.add_trace(go.Scatter(x=datetime_column, y=best_model.predict(df['DateTime'].values), mode='lines', line=dict(color='rgb(200, 2, 110)', width=2)))
            
            fig.add_vline(x=datetime_value, line_dash='dash', line_color='rgb(200, 2, 110)', line_width=1)
            
            fig.update_xaxes(
                tickvals=tickvals,
                ticktext=ticktext,
                title_text="Time",
                tickangle=90  # Add vertical rotation to x-axis labels
            )
            
            parameter_ = parameter.replace('/', '_')
            
            scale = set_y_scale(parameter)
            fig.update_yaxes(
                range=scale
            )
            
            fig.update_layout(
                xaxis_title='Time',
                yaxis_title=parameter,
                showlegend=False,
                margin=dict(l=10, r=10, t=10, b=10),
                font=dict(size=20)
            )
            
            fig.write_image(
                f'plots/305/{parameter_}.png',
                scale=3
            )
            
            # fig.show()

        else:
            
            br_value1 = round(best_model.get_results()['estimates']['breakpoint1']['estimate'], 6)
            br_value2 = round(best_model.get_results()['estimates']['breakpoint2']['estimate'], 6)
            datetime_value1 = datetime_column[(df['DateTime'] - br_value1).abs().idxmin()]
            datetime_value2 = datetime_column[(df['DateTime'] - br_value2).abs().idxmin()]
            
            print(f'R2: {r2_scores.max()}')
            print(f'Breakpoint 1: {datetime_value1}')
            print(f'Breakpoint 2: {datetime_value2}')

            fig = go.Figure()
            
            fig.add_trace(go.Scatter
                            (x=datetime_column, y=df[parameter], mode='lines',line=dict(color='black', width=1.5), opacity=0.5))
            fig.add_trace(go.Scatter
                            (x=datetime_column, y=best_model.predict(df['DateTime'].values), mode='lines', line=dict(color='rgb(200, 2, 110)', width=2)))
            
            fig.add_vline(x=datetime_value1, line_dash='dash', line_color='rgb(200, 2, 110)', line_width=1)
            fig.add_vline(x=datetime_value2, line_dash='dash', line_color='rgb(200, 2, 110)', line_width=1)
            
            fig.update_xaxes(
                tickvals=tickvals,
                ticktext=ticktext,
                title_text="Time",
                tickangle=90  # Add vertical rotation to x-axis labels
            )
            
            scale = set_y_scale(parameter)
            
            fig.update_yaxes(
                range=scale
            )
            
            fig.update_layout(
                xaxis_title='Time',
                yaxis_title=parameter,
                showlegend=False,
                margin=dict(l=10, r=10, t=10, b=10),
                font=dict(size=20)
            )
            
            parameter_ = parameter.replace('/', '_')
            
            fig.write_image(
                f'plots/305/{parameter_}.png',
                scale=3
            )
            
            # fig.show()

## Compute Trend based on Breakpoints Splits and Perform MK test

In [ ]:
mk_results_305_df = pd.DataFrame(
    index=columns,
    columns=pd.MultiIndex.from_product([['Before', 'After'], ['MK Test', 'P-Value']])
)

for parameter in results_305_df.index:
    
    r2_scores = results_305_df.loc[parameter, (slice(None), 'R2 Adj')]
    
    r2_scores = r2_scores.apply(lambda x: -np.inf if isinstance(x, str) else x)
    best_model_index = r2_scores.idxmax()[0]
    
    best_model = results_305_df.loc[parameter, (best_model_index, 'Model')]
    
    df = station_305_df[['DateTime', parameter]].copy()
    df.dropna(inplace=True)
    
    df['stdDateTime'] = df['DateTime'].astype(int)
    df['stdDateTime'] = scaler.fit_transform(df[['stdDateTime']])
    
    # modify df for Prophet
    df.rename(
        columns={
            'DateTime': 'ds',
            parameter: 'y'
        },
        inplace=True
    )
        
    # piecewise model
    if best_model_index == 1:
    
        br_value = round(best_model.get_results()['estimates']['breakpoint1']['estimate'], 6)
        datetime_value = df.loc[(df['stdDateTime'] - br_value).abs().idxmin()]['ds']
        
        before_df = df[df['ds'] < datetime_value]
        after_df = df[df['ds'] >= datetime_value]
        
        before_df.reset_index(drop=True, inplace=True)
        after_df.reset_index(drop=True, inplace=True)
        
        before_model = Prophet()
        before_model.fit(before_df[['ds', 'y']])
        
        before_future = before_model.make_future_dataframe(periods=0)
        before_forecast = before_model.predict(before_future)
        
        after_model = Prophet()
        after_model.fit(after_df)
        
        before_future = before_model.make_future_dataframe(periods=0)
        before_forecast = before_model.predict(before_future)
        
        after_future = after_model.make_future_dataframe(periods=0)
        after_forecast = after_model.predict(after_future)
        
        before_mk = mk.original_test(before_df['y'] - before_forecast['yearly'])
        after_mk = mk.original_test(after_df['y'] - after_forecast['yearly'])
        
        mk_results_305_df.loc[parameter, ('Before', 'MK Test')] = before_mk.trend
        mk_results_305_df.loc[parameter, ('Before', 'P-Value')] = before_mk.p
        
        
        mk_results_305_df.loc[parameter, ('After', 'MK Test')] = after_mk.trend
        mk_results_305_df.loc[parameter, ('After', 'P-Value')] = after_mk.p

    elif best_model_index == 2:
        
        br_value1 = round(best_model.get_results()['estimates']['breakpoint1']['estimate'], 6)
        br_value2 = round(best_model.get_results()['estimates']['breakpoint2']['estimate'], 6)
        datetime_value1 = df.loc[(df['stdDateTime'] - br_value1).abs().idxmin()]['ds']
        datetime_value2 = df.loc[(df['stdDateTime'] - br_value2).abs().idxmin()]['ds']

        # choose datetime_value based on parameter
        if parameter == 'DOC (mg/l)':
            value = datetime_value1
        if parameter == 'Sulphate (mg/l)':
            value = datetime_value2
        if parameter == 'Flow River Rate (m³/s)':
            value = datetime_value2
        if parameter == 'Ammonium (mg/l)':
            value = datetime_value2
        if parameter == 'Nitrate (mg/l)':
            value = datetime_value1
        if parameter == 'Conductivity (µS/cm)':
            value = datetime_value2
        if parameter == 'pH':
            value = datetime_value1
            
        before_df = df[df['ds'] < value]
        after_df = df[df['ds'] >= value]
        
        before_df.reset_index(drop=True, inplace=True)
        after_df.reset_index(drop=True, inplace=True)
        
        before_model = Prophet()
        before_model.fit(before_df[['ds', 'y']])
        
        after_model = Prophet()
        after_model.fit(after_df[['ds', 'y']])
        
        before_future = before_model.make_future_dataframe(periods=0)
        before_forecast = before_model.predict(before_future)
        
        after_future = after_model.make_future_dataframe(periods=0)
        after_forecast = after_model.predict(after_future)
        
        # perform MK test on the trend
        before_mk = mk.original_test(before_df['y'] - before_forecast['yearly'])
        after_mk = mk.original_test(after_df['y'] - after_forecast['yearly'])
        
        mk_results_305_df.loc[parameter, ('Before', 'MK Test')] = before_mk.trend
        mk_results_305_df.loc[parameter, ('Before', 'P-Value')] = before_mk.p
        
        mk_results_305_df.loc[parameter, ('After', 'MK Test')] = after_mk.trend
        mk_results_305_df.loc[parameter, ('After', 'P-Value')] = after_mk.p

In [ ]:
mk_results_305_df